# Pseudobulk Preprocessing for Anti-GAD65 Immune Cell Profiling

This notebook performs pseudobulk RNA-seq analysis on T-cell receptor (TCR) data from cerebrospinal fluid (CSF) and peripheral blood mononuclear cells (PBMC) samples, comparing expanded versus non-expanded clones.

## Overview
- **Input**: AnnData object with integrated single-cell RNA-seq data
- **Analysis**: Differential expression analysis using PyDESeq2
- **Output**: CSV files with differential expression results

## Requirements
Key packages:
- scanpy
- pandas
- numpy
- pydeseq2

## 1. Import Libraries

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

sc.settings.verbosity = 1

## 2. Define Helper Functions

In [ ]:
def create_pseudobulk(adata, sample_col='sample', condition_col='expansion'):
    """
    Create pseudobulk samples by aggregating counts across cells.
    
    Parameters:
    -----------
    adata : AnnData
        Annotated data object containing single-cell expression data
    sample_col : str
        Column name in adata.obs containing sample identifiers
    condition_col : str
        Column name in adata.obs containing condition labels
    
    Returns:
    --------
    AnnData
        Pseudobulk AnnData object with aggregated counts
    """
    pseudobulk_samples = []
    
    for sample_id in adata.obs[sample_col].unique():
        adata_sample = adata[adata.obs[sample_col] == sample_id]
        
        for condition in adata_sample.obs[condition_col].unique():
            adata_condition = adata_sample[adata_sample.obs[condition_col] == condition]
            
            pseudobulk_counts = adata_condition.X.sum(axis=0)
            pseudobulk_obj = sc.AnnData(
                X=pseudobulk_counts,
                var=adata_condition.var[[]]
            )
            
            pseudobulk_obj.obs_names = [f"{sample_id}_{condition}"]
            pseudobulk_obj.obs[condition_col] = condition
            
            pseudobulk_samples.append(pseudobulk_obj)
    
    return sc.concat(pseudobulk_samples)


def run_deseq2_analysis(pseudobulk_adata, design_factor='expansion', 
                        contrast_levels=('expansion', 'non-expansion')):
    """
    Perform differential expression analysis using DESeq2.
    
    Parameters:
    -----------
    pseudobulk_adata : AnnData
        Pseudobulk AnnData object
    design_factor : str
        Design factor for differential expression
    contrast_levels : tuple
        Levels to contrast (reference, comparison)
    
    Returns:
    --------
    pd.DataFrame
        Differential expression results sorted by test statistic
    """
    count_matrix = pd.DataFrame(
        pseudobulk_adata.X, 
        columns=pseudobulk_adata.var_names
    ).astype(int)
    
    dds = DeseqDataSet(
        counts=count_matrix,
        metadata=pseudobulk_adata.obs,
        design_factors=design_factor
    )
    
    dds.deseq2()
    
    deseq_stats = DeseqStats(
        dds, 
        contrast=(design_factor, contrast_levels[0], contrast_levels[1])
    )
    deseq_stats.summary()
    
    results_df = deseq_stats.results_df.sort_values('stat', ascending=False)
    
    return results_df

## 3. Load Data

Load the preprocessed AnnData object containing integrated single-cell RNA-seq data from GAD65-specific T cells.

In [ ]:
# Update this path to your data file location
INPUT_FILE = 'seurat_object_combined_singlets_integrated_GAD_CSF_PBMC_Tcell_TCR_transfered_ADATA_filtered.h5ad'

adata = sc.read_h5ad(INPUT_FILE)
print(f"Loaded AnnData with {adata.n_obs} cells and {adata.n_vars} genes")

---
# Analysis 1: CSF Compartment (Expanded vs Non-Expanded Clones)

## 3.1. Subset CSF Samples

In [ ]:
adata_csf = adata[adata.obs['Sampletype'].isin(['CSF'])].copy()
print(f"CSF subset: {adata_csf.n_obs} cells")
print(f"Max count value: {adata_csf.X.max()}")

## 3.2. Create Pseudobulk Samples

In [ ]:
pseudobulk_csf = create_pseudobulk(
    adata_csf, 
    sample_col='sample', 
    condition_col='expansion'
)

print(f"Created {pseudobulk_csf.n_obs} pseudobulk samples")
display(pseudobulk_csf.obs)

## 3.3. Run Differential Expression Analysis

In [ ]:
deseq_results_csf = run_deseq2_analysis(
    pseudobulk_csf,
    design_factor='expansion',
    contrast_levels=('expansion', 'non-expansion')
)

print(f"\nTop 10 differentially expressed genes:")
display(deseq_results_csf.head(10))

## 3.4. Save Results

In [ ]:
OUTPUT_CSF = 'pseudobulk_deseq2_GAD_CSF_expanded_vs_nonexpanded.csv'
deseq_results_csf.to_csv(OUTPUT_CSF)
print(f"Results saved to: {OUTPUT_CSF}")

---
# Analysis 2: PBMC Compartment (Expanded vs Non-Expanded Clones)

## 4.1. Load and Subset PBMC Samples

In [ ]:
adata = sc.read_h5ad(INPUT_FILE)
adata_pbmc = adata[adata.obs['Sampletype'].isin(['PBMC'])].copy()
print(f"PBMC subset: {adata_pbmc.n_obs} cells")
print(f"Max count value: {adata_pbmc.X.max()}")

## 4.2. Create Pseudobulk Samples

In [ ]:
pseudobulk_pbmc = create_pseudobulk(
    adata_pbmc,
    sample_col='sample',
    condition_col='expansion'
)

print(f"Created {pseudobulk_pbmc.n_obs} pseudobulk samples")
display(pseudobulk_pbmc.obs)

## 4.3. Run Differential Expression Analysis

In [ ]:
deseq_results_pbmc = run_deseq2_analysis(
    pseudobulk_pbmc,
    design_factor='expansion',
    contrast_levels=('expansion', 'non-expansion')
)

print(f"\nTop 10 differentially expressed genes:")
display(deseq_results_pbmc.head(10))

## 4.4. Save Results

In [ ]:
OUTPUT_PBMC = 'pseudobulk_deseq2_GAD_PBMC_expanded_vs_nonexpanded.csv'
deseq_results_pbmc.to_csv(OUTPUT_PBMC)
print(f"Results saved to: {OUTPUT_PBMC}")

## 4.5. Statistical Visualization with R

The differential expression results are visualized in R for generating volcano plots.

---
## Session Information

In [ ]:
sc.logging.print_versions()